In [1]:
import pandas as pd
import numpy as np
import math
import tensorflow as tf

In [2]:
df = pd.DataFrame()
print(-math.pi)

-3.141592653589793


In [3]:
print(np.sin(math.pi))

1.2246467991473532e-16


In [4]:
theta1= np.linspace(0,math.pi, 5500)
np.random.shuffle(theta1)
theta2= np.linspace(0,2*math.pi, 5500)
np.random.shuffle(theta2)

In [5]:
input_y= np.sin(theta1)+np.sin(theta1+theta2)
input_x= np.cos(theta1)+ np.cos(theta1+theta2)

In [6]:
df['X']=input_x
df['Y']=input_y
df['Theta1']=theta1
df['Theta2']=theta2
df.head(10)

,X,Y,Theta1,Theta2
0,0.331966,0.537454,2.266928,3.784308
1,0.774452,-0.103253,1.036914,3.944273
2,1.022089,1.030591,0.030850,1.517380
3,-0.764758,-0.033664,2.007557,2.356052
4,0.379295,0.074841,1.571082,3.530650
5,-1.319567,1.392803,2.615423,5.710740
6,1.339412,0.257141,1.010063,4.642404
7,-0.708800,1.788375,1.671060,0.554163
8,-1.800067,0.749983,2.970773,5.835284
9,1.036135,0.810567,1.516808,4.577276


In [7]:
df.to_csv('Inverse_Kinematic_Dataset.csv',header=False,index=False)

In [8]:
raw_df = np.loadtxt('Inverse_Kinematic_Dataset.csv' , delimiter=',')

In [9]:
inputs = raw_df[:,:2]
targets = raw_df[:,2:]
print(inputs.shape,targets.shape)

(5500, 2) (5500, 2)


In [10]:
inputs[3],targets[3]

(array([-0.76475826, -0.03366366]), array([2.00755712, 2.35605166]))

In [11]:
samples_count = inputs.shape[0]

train_samples_count = int(0.8*samples_count)
validation_samples_count = int(0.1*samples_count)
test_samples_count = samples_count - train_samples_count - validation_samples_count

train_inputs = inputs[:train_samples_count]
train_targets = targets[:train_samples_count]

validation_inputs = inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = targets[train_samples_count:train_samples_count+validation_samples_count]

test_inputs =inputs[train_samples_count+validation_samples_count:]
test_targets =targets[train_samples_count+validation_samples_count:]

In [12]:
np.savez('IK_data_train', inputs=train_inputs, targets=train_targets)
np.savez('IK_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('IK_data_test', inputs=test_inputs, targets=test_targets)

In [13]:
npz = np.load('IK_data_train.npz')
train_inputs, train_targets = npz['inputs'].astype(np.float64) , npz['targets'].astype(np.float64)


npz = np.load('IK_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.float64)


npz = np.load('IK_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float64), npz['targets'].astype(np.float64)

In [14]:
input_size = 2
output_size = 2
hidden_layer_size = 200

model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size+50, activation='tanh'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size+100, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size+150, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size+200, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size+250, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size+300, activation='tanh'),
    tf.keras.layers.Dense(hidden_layer_size+350, activation='tanh'),
    tf.keras.layers.Dense(output_size, activation='LeakyReLU') # output layer
])


# define the optimizer
# the loss function, 
# and the metrics-  obtaining at each iteration
model.compile(optimizer='adam', loss='MeanAbsoluteError', metrics=['accuracy'])

### Training

# set the batch size
batch_size = 10

#  maximum number of training epochs
max_epochs = 1000

early_stopping = tf.keras.callbacks.EarlyStopping(patience=3)

# fit the model
model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size,
          epochs=max_epochs,
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets),
          verbose = 2 
          )  

Epoch 1/1000
440/440 - 2s - loss: 0.9920 - accuracy: 0.8032 - val_loss: 0.8416 - val_accuracy: 0.8327
Epoch 2/1000
440/440 - 2s - loss: 0.8770 - accuracy: 0.8207 - val_loss: 0.8467 - val_accuracy: 0.8473
Epoch 3/1000
440/440 - 2s - loss: 0.8756 - accuracy: 0.8232 - val_loss: 0.8345 - val_accuracy: 0.8491
Epoch 4/1000
440/440 - 2s - loss: 0.8552 - accuracy: 0.8218 - val_loss: 0.8247 - val_accuracy: 0.8382
Epoch 5/1000
440/440 - 2s - loss: 0.8585 - accuracy: 0.8198 - val_loss: 0.8164 - val_accuracy: 0.8309
Epoch 6/1000
440/440 - 2s - loss: 0.8437 - accuracy: 0.8216 - val_loss: 0.8324 - val_accuracy: 0.8527
Epoch 7/1000
440/440 - 2s - loss: 0.8481 - accuracy: 0.8211 - val_loss: 0.8258 - val_accuracy: 0.8473
Epoch 8/1000
440/440 - 2s - loss: 0.8547 - accuracy: 0.8284 - val_loss: 0.8328 - val_accuracy: 0.8473


In [15]:
test_loss,test_accuracy=model.evaluate(test_inputs, test_targets)

18/18 [==============================] - 0s 2ms/step - loss: 0.8420 - accuracy: 0.8127


In [16]:
x=np.array([2*(np.cos(math.pi/3)), 2*(np.sin(math.pi/3))])
model.predict(x.reshape(1,2))

array([[0.8729464, 4.3039036]], dtype=float32)

In [17]:
theta1_test= np.linspace(0, math.pi, 5)
np.random.shuffle(theta1)
theta2_test= np.linspace(math.pi/2,2*math.pi, 5)
np.random.shuffle(theta2)
test_input_y= np.sin(theta1_test)+np.sin(theta1_test+theta2_test)
test_input_x= np.cos(theta1_test)+ np.cos(theta1_test+theta2_test)

In [18]:
df_test = pd.DataFrame()
df_test['X']=test_input_x
df_test['Y']=test_input_y
df_test['Theta1']=theta1_test
df_test['Theta2']=theta2_test
df_test.head(10)

,X,Y,Theta1,Theta2
0,1.000000,1.000000e+00,0.000000,1.570796
1,-0.216773,3.244233e-01,0.785398,2.748894
2,0.707107,2.928932e-01,1.570796,3.926991
3,-0.324423,1.630986e+00,2.356194,5.105088
4,-2.000000,4.898587e-16,3.141593,6.283185


In [19]:
df_test.to_csv('IK_Test_Dataset.csv',header=False,index=False)
test_df = np.loadtxt('IK_Test_Dataset.csv' , delimiter=',')

In [20]:
inputs_test = test_df[:,:2]
targets_test = test_df[:,2:]
print(inputs_test.shape,targets_test.shape)

(5, 2) (5, 2)


In [21]:
test_loss,test_accuracy=model.evaluate(inputs_test,targets_test)

1/1 [==============================] - 0s 29ms/step - loss: 1.4286 - accuracy: 0.8000
